In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer

from keras.models import Sequential
from keras.layers import Embedding,LSTM,SpatialDropout1D,Dense

pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

In [2]:
tweet=pd.read_csv('train.csv',encoding='latin-1',header=None)

In [3]:
tweet.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
tweet=tweet.iloc[:,[0,5]]

In [5]:
tweet.columns=['label','tweet']

In [6]:
tweet.head()

,label,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:
tweet.shape

(1600000, 2)

In [8]:
tweet[tweet.label==4].shape[0]/tweet.shape[0] #50% chrun rate good model condition

0.5

In [9]:
#check churn rate

tweet.label=tweet.label.apply(lambda x:1 if x==4 else 0)

In [10]:
tweet[tweet.label==1].shape[0]/tweet.shape[0] #50% chrun rate good model condition

0.5

In [11]:
tweet.label.value_counts()

0    800000
1    800000
Name: label, dtype: int64

# Step 1 : Data Preprocessing 

In [12]:
tweet.tweet=tweet['tweet'].apply(lambda x:x.lower())

In [13]:
tweet.tweet[10:30]

10          spring break in plain city... it's snowing 
11                           i just re-pierced my ears 
12    @caregiving i couldn't bear to watch it.  and ...
13    @octolinz16 it it counts, idk why i did either...
14    @smarrison i would've been the first, but i di...
15    @iamjazzyfizzle i wish i got to watch it with ...
16    hollis' death scene will hurt me severely to w...
17                                 about to file taxes 
18    @lettya ahh ive always wanted to see rent  lov...
19    @fakerpattypattz oh dear. were you drinking ou...
20    @alydesigns i was out most of the day so didn'...
21    one of my friend called me, and asked to meet ...
22     @angry_barista i baked you a cake but i ated it 
23               this week is not going as i had hoped 
24                           blagh class at 8 tomorrow 
25       i hate when i have to call and wake people up 
26    just going to cry myself to sleep after watchi...
27                               im sad now  mis

In [14]:
tweet.tweet[29]

'meh... almost lover is the exception... this track gets me depressed every time. '

In [15]:
# removing punctuation and other words
tweet['tweet'] = tweet['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [16]:
tweet.tweet[29]

'meh almost lover is the exception this track gets me depressed every time '

In [17]:
#Stemming the words will be done after tokenizeing
# lete first do remove stopwords

import nltk 
from nltk.corpus import stopwords

stopwords=stopwords.words('english')

In [18]:
tweet.tweet[29]

'meh almost lover is the exception this track gets me depressed every time '

In [19]:
# t='meh almost lover is the exception this track gets me depressed every time '

# words_t=[word for word in t.split() if word not in stopwords]
# print(' '.join(words_t))



In [20]:
tweet.tweet=tweet.tweet.apply(lambda x:' '.join([word for word in x.split() if word not in stopwords]))

In [21]:
tweet.tweet[29]

'meh almost lover exception track gets depressed every time'

In [22]:
# nltk.download('stopwords')
  

In [23]:
## removing stemming 

ps=PorterStemmer()

tweet.tweet=tweet.tweet.apply(lambda x:' '.join([ps.stem(word) for word in x.split()]))

In [24]:
tweet.tweet[29]

'meh almost lover except track get depress everi time'

In [25]:
# t='meh almost lover is the exception this track gets me depressed every time '

# print(' '.join([ps.stem(word) for word in t.split()]))

In [26]:
#pip install nltk

### Word tokenizer 

In [27]:
max_features=2000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(tweet.tweet.values)

In [28]:
#padding the values 
X = tokenizer.texts_to_sequences(tweet['tweet'].values)
X = pad_sequences(X)

In [29]:
y=tweet.label

# Model Building 

**Train test split**

In [30]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,
                                              random_state=42,
                                              stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1120000, 27)
(480000, 27)
(1120000,)
(480000,)


**Random Forest Classifier**

In [31]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=100,max_depth=None,min_samples_split=2,min_samples_leaf=1)

rfc.fit(X_train,y_train)

RandomForestClassifier()

In [32]:
y_train_pred=rfc.predict(X_train)
y_test_pred=rfc.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score

print('training accuracy score',accuracy_score(y_train_pred,y_train))
print('testing accuracy score',accuracy_score(y_test_pred,y_test))

training accuracy score 0.9714741071428571
testing accuracy score 0.6509229166666667


**XG Boost Classifier**

In [34]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [35]:
import xgboost as xgb
xgbc=xgb.XGBClassifier()
xgbc.fit(X_train,y_train)

C:\Users\Admin\anaconda3\envs\with tensorflow\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:03:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [36]:
y_train_pred=xgbc.predict(X_train)
y_test_pred=xgbc.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score

print('training accuracy score',accuracy_score(y_train_pred,y_train))
print('testing accuracy score',accuracy_score(y_test_pred,y_test))

training accuracy score 0.7144098214285715
testing accuracy score 0.7069854166666667


**Neural networks w/o LSTM layer**

In [38]:
#Create model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout,GlobalMaxPooling1D
from keras.models import Model
embed_dim = 128
lstm_out = 56
max_fatures = 20000


model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=128,input_length=X.shape[1]))
#model.add(LSTM(units=60,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.1))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 128)           2560000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 102       
                                                        

In [39]:
#training the model 
# batch_size=32

# model.fit(X_train,y_train,batch_size=32,epochs=2,verbose=1,validation_data=(X_test,y_test))

**Neural Networkl Model with LSTM layer**

In [40]:
# #Create model
# from keras.layers import Input, Dense, LSTM, Embedding, Dropout,GlobalMaxPooling1D
# from keras.models import Model

# max_feature=2000

# embed_dim=128
# lstm_out=196

# model=Sequential()
# model.add(Embedding(max_feature,embed_dim,input_length=X.shape[1]))
# model.add(Dropout(0.4))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())

In [41]:
#pip install tensorflow --upgrade

**We will do this letter as there seems some error related to LSTM Lib**

# Sentiment Analysis with BERT

In [42]:
#pip install transformers

In [43]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [44]:
#pip install ipywidgets

In [45]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [47]:
#fitting the model 
#training the model 
batch_size=32

model.fit(X_train,y_train,batch_size=32,epochs=2,verbose=1,validation_data=(X_test,y_test))

Epoch 1/2
    6/35000 [..............................] - ETA: 193:32:11 - loss: 0.7124 - accuracy: 0.5208

KeyboardInterrupt: 